# Autodoc

In [11]:
%pip install -q transformers accelerate

Note: you may need to restart the kernel to use updated packages.


In [1]:
%env TRANSFORMERS_CACHE /mnt/efs/hf/cache/
%env HF_CACHE /mnt/efs/hf/cache/

env: TRANSFORMERS_CACHE=/mnt/efs/hf/cache/
env: HF_CACHE=/mnt/efs/hf/cache/


In [2]:
import torch
torch.__version__

'2.0.0'

In [3]:
torch.cuda.is_available()

True

In [4]:
import bitsandbytes


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/envs/starcoder/lib/python3.8/site-packages/bitsandbytes-0.38.1-py3.8.egg/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /opt/conda/envs/starcoder/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/envs/starcoder/lib/python3.8/site-packages/bitsandbytes-0.38.1-py3.8.egg/bitsandbytes/libbitsandbytes_cuda117.so...


In [5]:
bitsandbytes.__path__

['/opt/conda/envs/starcoder/lib/python3.8/site-packages/bitsandbytes-0.38.1-py3.8.egg/bitsandbytes']

In [6]:
from bitsandbytes.cextension import CUDASetup
from bitsandbytes.cuda_setup.main import find_cuda_lib_in

cudaext = CUDASetup.get_instance()

In [7]:
import os

cudaext.run_cuda_setup()
cudaext.generate_instructions()

lib_ld_cuda_libs = find_cuda_lib_in(os.environ["LD_LIBRARY_PATH"])
lib_ld_cuda_libs


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/envs/starcoder/lib/python3.8/site-packages/bitsandbytes-0.38.1-py3.8.egg/bitsandbytes/libbitsandbytes_cuda117.so


{PosixPath('/usr/local/cuda/lib64/libcudart.so'),
 PosixPath('/usr/local/cuda/lib64/libcudart.so.11.0'),
 PosixPath('/usr/local/cuda/targets/x86_64-linux/lib/libcudart.so'),
 PosixPath('/usr/local/cuda/targets/x86_64-linux/lib/libcudart.so.11.0')}

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigcode/starcoder"
device = "cpu" # for GPU usage or "cpu" for CPU usage

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
device_map = {
    "transformer.word_embeddings": 0,
    "transformer.word_embeddings_layernorm": 0,
    "lm_head": "cpu",
    "transformer.h": 0,
    "transformer.wte.weight": 0,
    "transformer.wpe.weight": 0,
    "transformer.ln_f": 0,
}

quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                            device_map="auto", 
                                            #  device_map=device_map,
                                            #  quantization_config=quantization_config,
                                             load_in_8bit=True,
                                             torch_dtype=torch.bfloat16,
                                             #load_in_8bit_fp32_cpu_offload=True,
                                             trust_remote_code=True)

Overriding torch_dtype=torch.bfloat16 with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.
                        

In [ ]:
print(model.get_memory_footprint())

In [ ]:
inputs = tokenizer.encode("def print_hello_world():", return_tensors="pt").to(device)
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))